# 라이브러리 불러오기

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from selenium import webdriver
import time
from openpyxl import Workbook
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager

from pororo import Pororo
import sys
sys.path.insert(0, '../')
import soynlp
from soynlp.normalizer import *

print(soynlp.__version__)

In [ ]:
# 셀레니움 옵션 설정
wb = Workbook(write_only=True)
ws = wb.create_sheet()

options = webdriver.ChromeOptions() 
options.add_argument('headless') # 크롬 띄우는 창 없애기
options.add_argument('window-size=1920x1080') 
options.add_argument("disable-gpu") #그래픽 성능 낮춰서 크롤링 성능 높이기 
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36") # 네트워크 설정 
options.add_argument("lang=ko_KR") # 사이트 주언어 
driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options) 

In [ ]:
# Pororo 라이브러리 불러오기
sa = Pororo(task="sentiment", model="brainbert.base.ko.nsmc", lang="ko")
sh = Pororo(task="sentiment", model="brainbert.base.ko.shopping", lang="ko")

# 2021-8~2022-2 유튜브 리스트 크롤링

In [ ]:
# DB.xlsx에서 URL리스트와 채널리스트 불러오기
ytb_pd = pd.read_excel("DB.xlsx")
def plus_str(text):
    return text + "/videos"
url_list = ytb_pd.iloc[:,1].apply(plus_str).to_list()
channel_list = ytb_pd.iloc[:,3].to_list()
ytb_dict = dict(zip(channel_list,url_list))

In [ ]:
# 유튜브 리스트 크롤링
df = pd.DataFrame(columns=['채널명', '비디오_url', "텍스트"]) # 유튜브 리스트를 저장할 데이터 프레임
cnt = 0 # 작업진행도 확인을 위한 변수
for channel, url in ytb_dict.items():
    driver.get(url) 
    driver.implicitly_wait(3)
    time.sleep(1.5)
    driver.execute_script("window.scrollTo(0, 800)")
    time.sleep(3)

    # 페이지 끝까지 스크롤
    last_height = driver.execute_script("return document.documentElement.scrollHeight")

    while True:
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
        time.sleep(1.5)

        new_height = driver.execute_script("return document.documentElement.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    time.sleep(1.5)
    
    # 정보 추출하기
    html_source = driver.page_source
    soup = BeautifulSoup(html_source, 'html.parser')

    video_list = soup.select("h3>a")
    for video in video_list:
        aria = video.get('aria-label')
        base_url = "https://www.youtube.com"
        video_url = base_url + video.get('href')
        df = df.append(pd.DataFrame([[channel, video_url, aria]], columns=['채널명', '비디오_url', "텍스트"]))
    
    # 작업진행도 확인
    cnt += 1
    print("50개 중", cnt, "개 완료", channel, "까지 완료")

df = df.reset_index()
df = df.iloc[:,1:]

## 유튜브 리스트 전처리

In [ ]:
# 텍스트에서 날짜 분리하는 함수
def check(text):
  text = text.split("게시자: " + channel)[1]
  if "스트리밍" in text:
    temp = text.split("스트리밍 시간: ")[1].split()
  else:
    temp = text.split()
  return temp[0]

In [ ]:
# 날짜 컬럼 만들기
df["날짜"] = None
channels = df['채널명'].unique()
for channel in channels:
  df.loc[df['채널명']== channel, "날짜"] = df.loc[df['채널명']== channel, "텍스트"].apply(check)

In [ ]:
# 2021-08~2022-02 날짜 지정
df = df[(df["날짜"] == '3주') | (df["날짜"] == '1개월') | (df["날짜"] == '2개월') | (df["날짜"] == '3개월') | 
        (df["날짜"] == '4개월')| (df["날짜"] == '5개월') | (df["날짜"] == '6개월') | (df["날짜"] == '7개월')]

In [ ]:
# 사용할 컬럼만 남기기
df = df[["채널명", "비디오_url", "날짜"]]
df.shape

(2656, 3)

In [ ]:
# 팬심을 사용하는 데이터 확인하기 위한 함수
def check_fan(text):
  if text in fan:
    return "o"
  else:
    return "x"

In [ ]:
df['팬심사용여부'] = None

fan = ['나윤', '망득이', '체리아씨 [무편집풀영상]', 'Vtuber고야', '김씨할매', 
       '먹체토','초하이텐션 잉싸', '권율', "체리쉬_Cherry'sh", '플루트레미 Flute Remi', 'MIJU 미주',
       '캄나', '어써미채널', 'Singing Lfreda', '새담', '윈드아재 콘솔게임전문채널', '벼보리',
       'ONTIME 온타임', '재미있다 유튜브', '인간 김위칭', '갱맘TV', '임선비'] # 팬심 셀럽 리스트 

# 팬심을 사용할 경우 'o', 사용하지 않을 경우 'x'
df['팬심사용여부'] = df['채널명'].apply(check_fan)
fan_df = df[df['팬심사용여부'] == 'o']
print(fan_df.shape)
nofan_df = df[df['팬심사용여부'] == 'x']
print(nofan_df.shape)

(887, 4)
(1769, 4)


In [ ]:
# 확인 및 보관용 csv 파일저장
fan_df.to_csv('ytb_list_fan.csv', index=False, encoding="utf-8-sig")
nofan_df.to_csv('ytb_list_nofan.csv', index=False, encoding="utf-8-sig")

# 유튜브 댓글 크롤링

In [ ]:
# 팬심을 사용하는 셀럽 유튜브 댓글 트롤링
df = pd.DataFrame(columns=['채널명', '댓글', "날짜"]) # 유튜브 댓글을 저장할 데이터 프레임

for i in range(len(fan_df)):

    channel  = fan_df.iloc[i,0]
    URL = fan_df.iloc[i,1]

    driver.get(URL)
    driver.implicitly_wait(2)

    driver.execute_script("window.scrollTo(0, 800)")
    time.sleep(2)

    # 페이지 끝까지 스크롤
    last_height = driver.execute_script("return document.documentElement.scrollHeight")

    while True:
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
        time.sleep(1)

        new_height = driver.execute_script("return document.documentElement.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    time.sleep(1)

    # 정보 추출하기
    html_source = driver.page_source
    soup = BeautifulSoup(html_source, 'html.parser')

    comment_list = soup.select("yt-formatted-string#content-text")
    date_list = soup.select("div#header-author > yt-formatted-string > a")


    for j in range(len(comment_list)):
        comment = comment_list[j].text.strip()

        date = date_list[j].text.strip()
        df = df.append(pd.DataFrame([[channel, comment, date]], columns=['채널명', '댓글', "날짜"]))
    # 작업진행도 확인
    print("887개 중", i, "개 완료", channel, "까지 완료")

df = df.reset_index()
df = df.iloc[:,1:]
print(df)
# 확인 및 보관용 csv 파일저장
df.to_csv('ytb_crawling_fan_전처리전.csv', index=False, encoding="utf-8-sig")

In [ ]:
# 팬심을 사용하지 않는 셀럽 유튜브 댓글 트롤링
df = pd.DataFrame(columns=['채널명', '댓글', "날짜"]) # 유튜브 댓글을 저장할 데이터 프레임

for i in range(len(nofan_df)):

    channel  = nofan_df.iloc[i,0]
    URL = nofan_df.iloc[i,1]

    driver.get(URL)
    driver.implicitly_wait(2)

    driver.execute_script("window.scrollTo(0, 800)")
    time.sleep(2)

    # 페이지 끝까지 스크롤
    last_height = driver.execute_script("return document.documentElement.scrollHeight")

    while True:
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
        time.sleep(1)

        new_height = driver.execute_script("return document.documentElement.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    time.sleep(1)

    # 정보 추출하기
    html_source = driver.page_source
    soup = BeautifulSoup(html_source, 'html.parser')

    comment_list = soup.select("yt-formatted-string#content-text")
    date_list = soup.select("div#header-author > yt-formatted-string > a")


    for j in range(len(comment_list)):
        comment = comment_list[j].text.strip()

        date = date_list[j].text.strip()
        df = df.append(pd.DataFrame([[channel, comment, date]], columns=['채널명', '댓글', "날짜"]))

    # 작업진행도 확인
    print("1769개 중", i, "개 완료", channel, "까지 완료")

df = df.reset_index()
df = df.iloc[:,1:]
print(df)
# 확인 및 보관용 csv 파일저장
df.to_csv('ytb_crawling_nofan_전처리전.csv', index=False, encoding="utf-8-sig")

# 크롤링 완료한 데이터 전처리 및 감성분석

## 팬심을 사용하는 데이터 전처리 및 감성분석

### 전처리

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/CP2_Fancim/ytb_crawling_fan_전처리전.csv')
df.head()

In [ ]:
# 날짜 데이터 처리를 위한 함수
def check(text):
  if ("일 전" in text) or ("2주 전" in text) or ("시간" in text) or ("8개월" in text)or ("분 전" in text):
    return None
  return text

In [ ]:
# 전처리
df['댓글'] = df['댓글'].apply(str)
df['날짜'] = df['날짜'].apply(check)
df.dropna(axis=0, inplace=True)

df = df.drop(df[df['댓글']==' '].index)
df = df.drop(df[df['댓글']==''].index)
df = df.drop(df[df['댓글'].str.len()==1].index)
df = df[df['댓글'].str.len()<512]

df['댓글'] = df['댓글'].apply(only_hangle)
df['댓글'] = df['댓글'].apply(repeat_normalize)

stop_words = ['ㄱ','ㄴ','ㄷ','ㄹ','ㅁ','ㅂ','ㅅ','ㅇ','ㅈ','ㅊ','ㅋ','ㅌ','ㅍ','ㅎ',
              'ㅏ','ㅑ','ㅓ','ㅕ','ㅜ','ㅠ','ㅡ','ㅣ',
              'ㄱㄱ','ㄴㄴ','ㄷㄷ','ㄹㄹ','ㅁㅁ','ㅂㅂ','ㅅㅅ','ㅇㅇ','ㅈㅈ','ㅊㅊ','ㅋㅋ','ㅌㅌ','ㅍㅍ','ㅎㅎ',
              'ㅏㅏ','ㅑㅑ','ㅓㅓ','ㅕㅕ','ㅜㅜ','ㅠㅠ','ㅡㅡ','ㅣㅣ',
              '\n']

for w in stop_words:
  df['댓글'] = df['댓글'].replace(w,'', regex=True)

In [ ]:
# 날짜 데이터 형식 변경
con = (df['날짜'] == '3주 전') | (df['날짜'] == '3 weeks ago (edited)')
df.loc[con, 'month'] = '2022-02'
con = (df['날짜'] == '1개월 전') | (df['날짜'] == '1개월 전(수정됨)')
df.loc[con, 'month'] = '2022-02'
con = (df['날짜'] == '2개월 전') | (df['날짜'] == '2개월 전(수정됨)')
df.loc[con, 'month'] = '2022-01'
con = (df['날짜'] == '3개월 전') | (df['날짜'] == '3개월 전(수정됨)')
df.loc[con, 'month'] = '2021-12'
con = (df['날짜'] == '4개월 전') | (df['날짜'] == '4개월 전(수정됨)')
df.loc[con, 'month'] = '2021-11'
con = (df['날짜'] == '5개월 전') | (df['날짜'] == '5개월 전(수정됨)')
df.loc[con, 'month'] = '2021-10'
con = (df['날짜'] == '6개월 전') | (df['날짜'] == '6개월 전(수정됨)')
df.loc[con, 'month'] = '2021-09'
con = (df['날짜'] == '7개월 전') | (df['날짜'] == '7개월 전(수정됨)')
df.loc[con, 'month'] = '2021-08'

### 감성분석

In [ ]:
# 0.7이상일때 부정적 판단을 위한 함수
def check_neg(dic):
    if dic['negative'] >= 0.7:
        return 'negative'
    else:
        return 'positive'

In [ ]:
# 감성분석 라이브러리 적용
df['labels_ns'] = df['댓글'].apply(sa, show_probs=True)
df['labels_sh'] = df['댓글'].apply(sh, show_probs=True)

df['labels_ns_threshold'] = df['labels_ns'].apply(check_neg)
df['labels_sh_threshold'] = df['labels_sh'].apply(check_neg)
df['both_labels'] = 'positive'
df.loc[(df['labels_ns_threshold']=='negative')&(df['labels_sh_threshold']=='negative'), 'both_labels'] = 'negative'

In [ ]:
# 사용할 컬럼만 남기기
df = df[['채널명', '댓글', 'month', 'both_labels']]

In [ ]:
# 확인 및 보관용 csv 파일저장
df.to_csv('fancim_ytb_labels.csv', index=False, encoding='utf-8-sig')

## 팬심을 사용하지 않는 데이터 전처리 및 감성분석

### 전처리

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/CP2_Fancim/ytb_crawling_nofan_전처리전.csv')
df.head()

In [ ]:
# 전처리
df['댓글'] = df['댓글'].apply(str)
df['날짜'] = df['날짜'].apply(check)
df.dropna(axis=0, inplace=True)

df = df.drop(df[df['댓글']==' '].index)
df = df.drop(df[df['댓글']==''].index)
df = df.drop(df[df['댓글'].str.len()==1].index)
df = df[df['댓글'].str.len()<512]

df['댓글'] = df['댓글'].apply(only_hangle)
df['댓글'] = df['댓글'].apply(repeat_normalize)

stop_words = ['ㄱ','ㄴ','ㄷ','ㄹ','ㅁ','ㅂ','ㅅ','ㅇ','ㅈ','ㅊ','ㅋ','ㅌ','ㅍ','ㅎ',
              'ㅏ','ㅑ','ㅓ','ㅕ','ㅜ','ㅠ','ㅡ','ㅣ',
              'ㄱㄱ','ㄴㄴ','ㄷㄷ','ㄹㄹ','ㅁㅁ','ㅂㅂ','ㅅㅅ','ㅇㅇ','ㅈㅈ','ㅊㅊ','ㅋㅋ','ㅌㅌ','ㅍㅍ','ㅎㅎ',
              'ㅏㅏ','ㅑㅑ','ㅓㅓ','ㅕㅕ','ㅜㅜ','ㅠㅠ','ㅡㅡ','ㅣㅣ',
              '\n']

for w in stop_words:
  df['댓글'] = df['댓글'].replace(w,'', regex=True)

In [ ]:
# 날짜 데이터 형식 변경
con = (df['날짜'] == '3주 전') | (df['날짜'] == '3 weeks ago (edited)')
df.loc[con, 'month'] = '2022-02'
con = (df['날짜'] == '1개월 전') | (df['날짜'] == '1개월 전(수정됨)')
df.loc[con, 'month'] = '2022-02'
con = (df['날짜'] == '2개월 전') | (df['날짜'] == '2개월 전(수정됨)')
df.loc[con, 'month'] = '2022-01'
con = (df['날짜'] == '3개월 전') | (df['날짜'] == '3개월 전(수정됨)')
df.loc[con, 'month'] = '2021-12'
con = (df['날짜'] == '4개월 전') | (df['날짜'] == '4개월 전(수정됨)')
df.loc[con, 'month'] = '2021-11'
con = (df['날짜'] == '5개월 전') | (df['날짜'] == '5개월 전(수정됨)')
df.loc[con, 'month'] = '2021-10'
con = (df['날짜'] == '6개월 전') | (df['날짜'] == '6개월 전(수정됨)')
df.loc[con, 'month'] = '2021-09'
con = (df['날짜'] == '7개월 전') | (df['날짜'] == '7개월 전(수정됨)')
df.loc[con, 'month'] = '2021-08'

### 감성분석

In [ ]:
# 감성분석 라이브러리 적용
df['labels_ns'] = df['댓글'].apply(sa, show_probs=True)
df['labels_sh'] = df['댓글'].apply(sh, show_probs=True)

df['labels_ns_threshold'] = df['labels_ns'].apply(check_neg)
df['labels_sh_threshold'] = df['labels_sh'].apply(check_neg)
df['both_labels'] = 'positive'
df.loc[(df['labels_ns_threshold']=='negative')&(df['labels_sh_threshold']=='negative'), 'both_labels'] = 'negative'

In [ ]:
# 사용할 컬럼만 남기기
df = df[['채널명', '댓글', 'month', 'both_labels']]

In [ ]:
# 확인 및 보관용 csv 파일저장
df.to_csv('non_fancim_ytb_labels_ver2.csv', index=False, encoding='utf-8-sig')